In [116]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [117]:
#데이터를 불러와

import numpy as np

rows = np.loadtxt("./lotto.csv", delimiter=",")
row_count = len(rows)
print(row_count)

#이번주 당첨번호
lucky_numbers=[13,17,18,20,42,45]
second_number=[41]

998


In [118]:
import numpy as np

# 당첨번호를 원핫인코딩벡터(ohbin)으로 변환
def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

# 원핫인코딩벡터(ohbin)를 번호로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers

In [119]:
numbers = rows[:, 1:7]
ohbins = list(map(numbers2ohbin, numbers))

x_samples = ohbins[0:row_count-1]
y_samples = ohbins[1:row_count]

#원핫인코딩으로 표시
print("ohbins")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

#번호로 표시
print("numbers")
print("X[0]: " + str(ohbin2numbers(x_samples[0])))
print("Y[0]: " + str(ohbin2numbers(y_samples[0])))

ohbins
X[0]: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
Y[0]: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
numbers
X[0]: [10, 23, 29, 33, 37, 40]
Y[0]: [9, 13, 21, 25, 32, 42]


In [120]:
#windows를 만들자

seq_len = 30
sequence_length = seq_len + 1

result=[]
for index in range(len(x_samples)-sequence_length):
    result.append(x_samples[index: index+sequence_length])

    

In [121]:
result = np.array(x_samples)

row = int(round(result.shape[0]*0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train,(x_train.shape[0], x_train.shape[1],-1))
y_train = train[:,-1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],-1))
y_test = result[row:,-1]

x_train.shape, x_test.shape


((897, 44, 1), (100, 44, 1))

In [122]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

#from keras.layers import Dense, Activation 
from keras.layers import LeakyReLU

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda

from sklearn.metrics import f1_score 
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

optimizer = Adam(0.005)
loss = Huber()

#모델을 컴파일 해보자
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(44,1)))
model.add(LSTM(64, return_sequences=False))
#model.add(Dense(128, activation='relu'))
model.add(Dense(128))

model.add(Dense(45, activation=LeakyReLU(alpha=0.1)))
#model.add(LeakyReLU(alpha=0.2))
model.compile(loss=loss, optimizer=optimizer, metrics=f1_m)
#model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=f1_m)

model.summary()


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_34 (LSTM)              (None, 44, 50)            10400     
                                                                 
 lstm_35 (LSTM)              (None, 64)                29440     
                                                                 
 dense_34 (Dense)            (None, 128)               8320      
                                                                 
 dense_35 (Dense)            (None, 45)                5805      
                                                                 
Total params: 53,965
Trainable params: 53,965
Non-trainable params: 0
_________________________________________________________________


In [124]:
#Training

model.fit(x_train, y_train,
         validation_data=(x_test, y_test),
         batch_size=15,
         epochs=50,
         callbacks = [tf.keras.callbacks.EarlyStopping(patience=15, monitor ='val_loss')])

Epoch 1/50
60/60 [==============================] - 1s 23ms/step - loss: 0.0624 - f1_m: 0.0000e+00 - val_loss: 0.0394 - val_f1_m: 0.0000e+00
Epoch 2/50
60/60 [==============================] - 1s 21ms/step - loss: 0.0625 - f1_m: 0.0000e+00 - val_loss: 0.0392 - val_f1_m: 0.0000e+00
Epoch 3/50
60/60 [==============================] - 1s 21ms/step - loss: 0.0623 - f1_m: 0.0000e+00 - val_loss: 0.0395 - val_f1_m: 0.0000e+00
Epoch 4/50
60/60 [==============================] - 1s 21ms/step - loss: 0.0616 - f1_m: 0.0000e+00 - val_loss: 0.0384 - val_f1_m: 0.0000e+00
Epoch 5/50
60/60 [==============================] - 1s 21ms/step - loss: 0.0612 - f1_m: 0.0000e+00 - val_loss: 0.0438 - val_f1_m: 0.0000e+00
Epoch 6/50
60/60 [==============================] - 1s 21ms/step - loss: 0.0621 - f1_m: 0.0000e+00 - val_loss: 0.0391 - val_f1_m: 0.0000e+00
Epoch 7/50
60/60 [==============================] - 1s 21ms/step - loss: 0.0616 - f1_m: 0.0000e+00 - val_loss: 0.0399 - val_f1_m: 0.0000e+00
Epoch 8/50
60

In [125]:
def gen_numbers_from_probability(nums_prob, exp_numb, inv_numb):

    ball_box = []#꺼낼 공 모음
    selected_balls = []#꺼낸 공 모음

#임시로 예상수를 넣어놔
    temp_numb = []
    temp_numb = nums_prob
    temp_exp = []
    temp_exp = exp_numb

#역수를 찾아서 하나는 넣을거야
    inv_ran = np.random.randint(len(inv_numb), size=1)[0]#랜덤번호 하나 뽑아
    inv_ball = inv_numb[inv_ran]#랜덤번호로 역수 리스트에서 하나 수 뽑아
    selected_balls.append(inv_ball)#그 수를 꺼낸공 모음에 넣어
    temp_exp.extend(inv_numb)

#제외수는 빼
    for i in range (len(temp_exp)):
        temp_numb[temp_exp[i]-1] = 0

    
#나온 확률에 10000을 곱해서 공을 만들고 그중에 뽑는거

    for n in range(45):
        ball_count = int(((temp_numb[n] * 10000)%1000)/10) #확률 제로면 안뽑을거야
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    
#6개 뽑으면 끝남
    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    return selected_balls



''' 얘네가 원래 꺼

    ball_box = []
    


    for i in range (len(exp_numb)):
        nums_prob[exp_numb[i]-1] = 0
    
    for n in range(45):
        ball_count = int(nums_prob[n] * 100)
        #ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    return selected_balls

'''

' 얘네가 원래 꺼\n\n    ball_box = []\n    \n\n\n    for i in range (len(exp_numb)):\n        nums_prob[exp_numb[i]-1] = 0\n    \n    for n in range(45):\n        ball_count = int(nums_prob[n] * 100)\n        #ball_count = int(nums_prob[n] * 100 + 1)\n        ball = np.full((ball_count), n+1) #1부터 시작\n        ball_box += list(ball)\n\n    selected_balls = []\n\n    while True:\n        \n        if len(selected_balls) == 6:\n            break\n        \n        ball_index = np.random.randint(len(ball_box), size=1)[0]\n        ball = ball_box[ball_index]\n\n        if ball not in selected_balls:\n            selected_balls.append(ball)\n\n    return selected_balls\n\n'

In [126]:
# 마지막 회차까지 학습한 모델로 다음 회차 추론
# 지난 회차 역수 구해(inverse_numbers)

print('receive numbers')

model.reset_states()

xs = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],-1))

ys_pred = model.predict_on_batch(xs)


list_numbers = []
inverse_numbers = []

for i in range(6):
    a = 46 - ohbin2numbers(y_samples[-1])[i]
    inverse_numbers.append(a)
inverse_numbers.sort()

#여기부터 제외수 만들기
expect_numbers=[]
'''
#평균*0.9 보다 작은 값은 제외수로 넣어
_sum = 0
_mean = 0
for x in ys_pred[0]:
    if x <= 0:
        x = 0
    else:
        _sum += x
_mean = _sum / len(ys_pred[0])

_expect = []
'''

temp1 = list(set(ohbin2numbers(y_samples[-1])) & set(ohbin2numbers(y_samples[-2])))
expect_numbers.extend(temp1)
expect_numbers.append(int(rows[-1,7] + 1))

_mean = np.mean(ys_pred[0])
_std = np.std(ys_pred[0])


for i in range(45):
    if ys_pred[0][i] <= (_mean - _std * 1.1):
        expect_numbers.append(i+1)

'''
#0보다 작은 값은 제외수로 넣어
for j in range(45):
    if ys_pred[0][j] <= 0:
        expect_numbers.append(j+1)
        ys_pred[0][j] = 0
'''

n=0
while n < 10:
#for n in range(100):
    numbers = gen_numbers_from_probability(ys_pred[0],expect_numbers, inverse_numbers)
    numbers.sort()
    if (len(list(set(numbers).intersection(ohbin2numbers(y_samples[-1])))) <= 1) and (len(list(set(numbers).intersection(inverse_numbers))) <=1):
        if len(list(set(numbers).intersection(lucky_numbers))) == 3: 
            print('{0} : {1} --- 5등 당첨'.format(n, numbers))
        elif len(list(set(numbers).intersection(lucky_numbers))) == 4: 
            print('{0} : {1} --- 4등 당첨'.format(n, numbers))
        elif len(list(set(numbers).intersection(lucky_numbers))) == 5: 
            print('{0} : {1} --- 3등 당첨'.format(n, numbers))
        elif len(list(set(numbers).intersection(lucky_numbers))) == 6: 
            print('{0} : {1} --- 1등 당첨'.format(n, numbers))
        elif len(list(set(numbers).intersection(lucky_numbers))) == 5 and len(list(set(numbers).intersection(second_number))) == 1: 
            print('{0} : {1} --- 2등 당첨'.format(n, numbers))
        
        else:
            print('{0} : {1}'.format(n, numbers))
        n = n + 1
    list_numbers.append(numbers)

receive numbers
0 : [3, 5, 11, 23, 25, 26]
1 : [12, 16, 17, 22, 28, 40]
2 : [2, 5, 13, 25, 28, 30]
3 : [9, 13, 21, 33, 40, 41]
4 : [16, 20, 22, 24, 26, 38]
5 : [2, 12, 27, 29, 31, 43]
6 : [1, 2, 24, 27, 36, 41]
7 : [4, 14, 16, 21, 25, 30]
8 : [2, 4, 6, 14, 24, 25]
9 : [5, 14, 26, 30, 37, 41]


In [127]:
# 88회부터 지금까지 1등부터 5등까지 상금의 평균낸다.
mean_prize = [ np.mean(rows[87:, 8]),
           np.mean(rows[87:, 9]),
           np.mean(rows[87:, 10]),
           50000, 5000]

print(mean_prize)

train_idx = (0, 800)
val_idx = (801, 900)
test_idx = (901, len(x_samples))

[2436204039.2502747, 57221317.72777168, 1450618.6882546651, 50000, 5000]


In [128]:
# 등수와 상금을 반환함
# 순위에 오르지 못한 경우에는 등수가 0으로 반환함
def calc_reward(true_numbers, true_bonus, pred_numbers):

    count = 0

    for ps in pred_numbers:
        if ps in true_numbers:
            count += 1

    if count == 6:
        return 0, mean_prize[0]
    elif count == 5 and true_bonus in pred_numbers:
        return 1, mean_prize[1]
    elif count == 5:
        return 2, mean_prize[2]
    elif count == 4:
        return 3, mean_prize[3]
    elif count == 3:
        return 4, mean_prize[4]

    return 5, 0

In [129]:
train_total_reward = []
train_total_grade = np.zeros(6, dtype=int)

val_total_reward = []
val_total_grade = np.zeros(6, dtype=int)

test_total_reward = []
test_total_grade = np.zeros(6, dtype=int)

model.reset_states()

'''
#지난회차 역수 inverse_numbers에 넣기
inverse_numbers = []
for i in range(6):
    a = 46 - ohbin2numbers(y_samples[-1])[i]
    inverse_numbers.append(a)
inverse_numbers.sort()
'''

print('[No. ] 1st 2nd 3rd 4th 5th 6th Rewards')


for i in range(len(x_samples)):
#    xs = x_samples[i].reshape(1, 1, 45)
    xs = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],-1))

    ys_pred = model.predict_on_batch(xs) # 모델의 출력값을 얻음
    
    
    list_numbers = []
    inverse_numbers = []

    for j in range(6):
        a = 46 - ohbin2numbers(y_samples[-1])[j]
        inverse_numbers.append(a)
    inverse_numbers.sort()

    #여기부터 제외수 만들기
    expect_numbers=[]
    # 확률 하위 16%는 제외수로 날려
    _mean = np.mean(ys_pred[0])
    _std = np.std(ys_pred[0])
    for k in range(45):
        if ys_pred[0][k] <= (_mean - _std):
            expect_numbers.append(k+1)
    
    '''
    #평균*0.9 보다 작은 값은 제외수로 넣어
    _sum = 0
    _mean = 0
    for x in ys_pred[0]:
        if x <= 0:
            x = 0
        else:
            _sum += x
    _mean = _sum / len(ys_pred[0])

    _expect = []

    for k in range(45):
        if ys_pred[0][k] <= (_mean * 0.985):
            expect_numbers.append(k+1)
    '''
    sum_reward = 0
    sum_grade = np.zeros(6, dtype=int) # 6등까지 변수

    for n in range(20): # 20판 수행
        numbers = gen_numbers_from_probability(ys_pred[0],expect_numbers, inverse_numbers)
          
        
        #i회차 입력 후 나온 출력을 i+1회차와 비교함
        grade, reward = calc_reward(rows[i+1,1:7], rows[i+1,7], numbers) 
        
        sum_reward += reward
        sum_grade[grade] += 1

        if i >= train_idx[0] and i < train_idx[1]:
            train_total_grade[grade] += 1
        elif i >= val_idx[0] and i < val_idx[1]:
            val_total_grade[grade] += 1
        elif i >= test_idx[0] and i < test_idx[1]:
            val_total_grade[grade] += 1
    
    if i >= train_idx[0] and i < train_idx[1]:
        train_total_reward.append(sum_reward)
    elif i >= val_idx[0] and i < val_idx[1]:
        val_total_reward.append(sum_reward)
    elif i >= test_idx[0] and i < test_idx[1]:
        test_total_reward.append(sum_reward)
                        
    print('[{0:4d}] {1:3d} {2:3d} {3:3d} {4:3d} {5:3d} {6:3d} {7:15,d}'.format(i+1, sum_grade[0], sum_grade[1], sum_grade[2], sum_grade[3], sum_grade[4], sum_grade[5], int(sum_reward)))

print('Total') 
print('==========')    
print('Train {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(train_total_grade[0], train_total_grade[1], train_total_grade[2], train_total_grade[3], train_total_grade[4], train_total_grade[5], int(sum(train_total_reward))))
print('Val   {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(val_total_grade[0], val_total_grade[1], val_total_grade[2], val_total_grade[3], val_total_grade[4], val_total_grade[5], int(sum(val_total_reward))))
print('Test  {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(test_total_grade[0], test_total_grade[1], test_total_grade[2], test_total_grade[3], test_total_grade[4], test_total_grade[5], int(sum(test_total_reward))))
print('==========')


[No. ] 1st 2nd 3rd 4th 5th 6th Rewards
[   1]   0   0   0   0   0  20               0
[   2]   0   0   0   0   0  20               0
[   3]   0   0   0   0   2  18          10,000
[   4]   0   0   0   1   1  18          55,000
[   5]   0   0   0   0   0  20               0
[   6]   0   0   0   0   2  18          10,000
[   7]   0   0   0   0   2  18          10,000
[   8]   0   0   0   0   0  20               0
[   9]   0   0   0   0   0  20               0
[  10]   0   0   0   0   0  20               0
[  11]   0   0   0   0   0  20               0
[  12]   0   0   0   0   0  20               0
[  13]   0   0   0   0   1  19           5,000
[  14]   0   0   0   0   0  20               0
[  15]   0   0   0   0   0  20               0
[  16]   0   0   0   0   0  20               0
[  17]   0   0   0   0   1  19           5,000
[  18]   0   0   0   0   2  18          10,000
[  19]   0   0   0   0   1  19           5,000
[  20]   0   0   0   0   0  20               0
[  21]   0   0   0   

In [113]:
ys_pred[0]

array([0.        , 0.        , 0.07603382, 0.07535477, 0.07750858,
       0.07431756, 0.08051202, 0.0849608 , 0.07941183, 0.07952076,
       0.07705883, 0.07604516, 0.07598406, 0.07600808, 0.        ,
       0.08046301, 0.07746305, 0.0792172 , 0.        , 0.07610177,
       0.07327504, 0.        , 0.07772245, 0.07427373, 0.        ,
       0.07572727, 0.0764158 , 0.07420748, 0.07333403, 0.        ,
       0.07415187, 0.        , 0.07613811, 0.07248724, 0.078926  ,
       0.        , 0.        , 0.07782727, 0.        , 0.07516908,
       0.08238313, 0.        , 0.07398169, 0.07697114, 0.07269738],
      dtype=float32)

In [114]:
expect_numbers = set(expect_numbers)
expect_numbers

{1, 2, 15, 19, 22, 25, 30, 32, 36, 37, 39, 42}

In [16]:
inverse_numbers

[2, 22, 30, 32, 39, 42]